In [ ]:
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import time, sys
import pickle
from scipy.io import savemat

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Choose either Re=1000 or Re=100
Re=10000

# load FOM solutions
if Re==10000:
    nx=60
    ny=60
    nt=1500
    
    FOM_solution=pickle.load(open('./data/FOM_high_Re.p','rb'))
elif Re==100:
    nx=60
    ny=60
    nt=1500
    
    FOM_solution=pickle.load(open('./data/FOM_low_Re.p','rb'))
else:
    raise NameError('{} is given for Re, but it must be either 100 or 10000'.format(Re))
    
u_full=FOM_solution['u']
v_full=FOM_solution['v']

u0=np.copy(u_full[0])
v0=np.copy(v_full[0])

# set parameters
nxy=(nx-2)*(ny-2)
dx=1/(nx-1)
dy=1/(ny-1)
dt=2/nt

I=sp.eye(nxy,format='csr')

# full indicies, free indicies, fixed indicies
[xv,yv]=np.meshgrid(np.linspace(0,1,nx),np.linspace(0,1,ny),indexing='xy')
x=xv.flatten()
y=yv.flatten()

multi_index_i,multi_index_j=np.meshgrid(np.arange(nx),np.arange(ny),indexing='xy')
full_multi_index=(multi_index_j.flatten(),multi_index_i.flatten())
free_multi_index=(multi_index_j[1:-1,1:-1].flatten(),multi_index_i[1:-1,1:-1].flatten())
x0_multi_index=(multi_index_j[1:-1,0].flatten(),multi_index_i[1:-1,0].flatten())
x1_multi_index=(multi_index_j[1:-1,-1].flatten(),multi_index_i[1:-1,-1].flatten())
y0_multi_index=(multi_index_j[0,1:-1].flatten(),multi_index_i[0,1:-1].flatten())
y1_multi_index=(multi_index_j[-1,1:-1].flatten(),multi_index_i[-1,1:-1].flatten())

dims=(ny,nx)
full_raveled_indicies=np.ravel_multi_index(full_multi_index,dims)
free_raveled_indicies=np.ravel_multi_index(free_multi_index,dims)
x0_raveled_indicies=np.ravel_multi_index(x0_multi_index,dims)
x1_raveled_indicies=np.ravel_multi_index(x1_multi_index,dims)
x01_raveled_indicies=np.concatenate((x0_raveled_indicies,x1_raveled_indicies))
y0_raveled_indicies=np.ravel_multi_index(y0_multi_index,dims)
y1_raveled_indicies=np.ravel_multi_index(y1_multi_index,dims)
y01_raveled_indicies=np.concatenate((y0_raveled_indicies,y1_raveled_indicies))
fixed_raveled_indicies=np.setdiff1d(full_raveled_indicies,free_raveled_indicies)

# boundary one-hot vector
x0_one_hot=np.eye(nx-2)[0]
y0_one_hot=np.eye(ny-2)[0]
x1_one_hot=np.eye(nx-2)[-1]
y1_one_hot=np.eye(ny-2)[-1]

# inner grid
inner_multi_index_i,inner_multi_index_j=np.meshgrid(np.arange(nx-2),np.arange(ny-2),indexing='xy')
inner_x_multi_index=(np.concatenate((inner_multi_index_j[:,0].flatten(),inner_multi_index_j[:,-1].flatten())),
                     np.concatenate((inner_multi_index_i[:,0].flatten(),inner_multi_index_i[:,-1].flatten())))
inner_y_multi_index=(np.concatenate((inner_multi_index_j[0,:].flatten(),inner_multi_index_j[-1,:].flatten())),
                     np.concatenate((inner_multi_index_i[0,:].flatten(),inner_multi_index_i[-1,:].flatten())))

inner_dims=(ny-2,nx-2)
inner_x_raveled_indicies=np.ravel_multi_index(inner_x_multi_index,inner_dims)
inner_y_raveled_indicies=np.ravel_multi_index(inner_y_multi_index,inner_dims)

# set FOM solution (free DoFs)
u_fom=u_full[1:,free_raveled_indicies]
v_fom=v_full[1:,free_raveled_indicies]

u_ref=u_full[0,free_raveled_indicies]
v_ref=v_full[0,free_raveled_indicies]

# nparam
nparamArr_LSROM=np.array([2,4,6,8,10])

# avg. rel. errors
relErr_LSROM_g=np.zeros((2,nparamArr_LSROM.shape[0]))
relErr_LSROM_lspg=np.zeros((2,nparamArr_LSROM.shape[0]))

# max. rel. errors
maxErr_LSROM_g=np.zeros((2,nparamArr_LSROM.shape[0]))
maxErr_LSROM_lspg=np.zeros((2,nparamArr_LSROM.shape[0]))

# proj. errors
proErr_LSROM=np.zeros((2,nparamArr_LSROM.shape[0]))
for ii in range(len(nparamArr_LSROM)):
    nparam=nparamArr_LSROM[ii]
    
    # load POD basis
    if Re==10000:
        file_name_POD="./model/POD_high_Re_nparam_{}.p".format(nparam)
    elif Re==100:
        file_name_POD="./model/POD_low_Re_nparam_{}.p".format(nparam)
    else:
        raise NameError('{} is given for Re, but it must be either 100 or 10000'.format(Re))

    POD = pickle.load(open(file_name_POD,'rb'))

    phi_u = POD['u']
    phi_v = POD['v']

    phi_uT=phi_u.T
    phi_vT=phi_v.T

    proj_u=(np.eye(u_fom.shape[1])-phi_u.dot(phi_uT)).dot((u_fom-u_ref).T)
    pro_err_u=np.sqrt(np.sum(np.linalg.norm(proj_u,ord=2,axis=0)**2))/np.sqrt(np.sum(np.linalg.norm(u_fom,ord=2,axis=1)**2))*100
    proErr_LSROM[0,ii]=pro_err_u
    
    proj_v=(np.eye(v_fom.shape[1])-phi_v.dot(phi_vT)).dot((v_fom-v_ref).T)
    pro_err_v=np.sqrt(np.sum(np.linalg.norm(proj_v,ord=2,axis=0)**2))/np.sqrt(np.sum(np.linalg.norm(v_fom,ord=2,axis=1)**2))*100
    proErr_LSROM[1,ii]=pro_err_v

In [ ]:
# first order derivative
# central
Mb=sp.diags([np.ones(nx-2),-np.ones(nx-2)],[0,-1],(nx-2,nx-2))
M=sp.kron(sp.eye(ny-2),Mb,format="csr")

Ib=sp.eye(nx-2)
N=sp.kron(sp.diags([np.ones(ny-2),-np.ones(ny-2)],[0,-1],(ny-2,ny-2)),Ib,format="csr")

# laplacian operator
Dxb=sp.diags([-2*np.ones(nx-2),np.ones(nx-2),np.ones(nx-2)],[0,-1,1],(nx-2,nx-2))
Dx=sp.kron(sp.eye(ny-2),Dxb,format="csr")

Ib=sp.eye(nx-2)
Dy=sp.kron(sp.diags([-2*np.ones(ny-2),np.ones(ny-2),np.ones(ny-2)],[0,-1,1],(ny-2,ny-2)),Ib,format="csr")

In [ ]:
# boundary for first order derivative term
Bdudx0_cur=np.kron(u0[x0_raveled_indicies],x0_one_hot)
Bdudy0_cur=np.kron(y0_one_hot,u0[y0_raveled_indicies])
Bdvdx0_cur=np.kron(v0[x0_raveled_indicies],x0_one_hot)
Bdvdy0_cur=np.kron(y0_one_hot,v0[y0_raveled_indicies])
Bdudx1_cur=np.kron(u0[x1_raveled_indicies],x1_one_hot)
Bdudy1_cur=np.kron(y1_one_hot,u0[y1_raveled_indicies])
Bdvdx1_cur=np.kron(v0[x1_raveled_indicies],x1_one_hot)
Bdvdy1_cur=np.kron(y1_one_hot,v0[y1_raveled_indicies])
    
# boundary for second order derivative term
bxu_cur=np.zeros(nxy)
byu_cur=np.zeros(nxy)
bxv_cur=np.zeros(nxy)
byv_cur=np.zeros(nxy)

bxu_cur[inner_x_raveled_indicies]=u0[x01_raveled_indicies]
byu_cur[inner_y_raveled_indicies]=u0[y01_raveled_indicies]
bxv_cur[inner_x_raveled_indicies]=v0[x01_raveled_indicies]
byv_cur[inner_y_raveled_indicies]=v0[y01_raveled_indicies]

In [ ]:
Mu_ref=M.dot(u_ref)
Nu_ref=N.dot(u_ref)
Mv_ref=M.dot(v_ref)
Nv_ref=N.dot(v_ref)

Dxu_ref=Dx.dot(u_ref)
Dyu_ref=Dy.dot(u_ref)
Dxv_ref=Dx.dot(v_ref)
Dyv_ref=Dy.dot(v_ref)

# Galerkin

In [ ]:
for ii in range(len(nparamArr_LSROM)):
    nparam=nparamArr_LSROM[ii]
    
    # load POD basis
    if Re==10000:
        POD_basis=5
        file_name_POD="./model/POD_high_Re_nparam_{}.p".format(nparam)
    elif Re==100:
        POD_basis=5
        file_name_POD="./model/POD_low_Re_nparam_{}.p".format(nparam)
    else:
        raise NameError('{} is given for Re, but it must be either 100 or 10000'.format(Re))

    POD = pickle.load(open(file_name_POD,'rb'))

    phi_u = POD['u']
    phi_v = POD['v']

    phi_u_pinv=np.linalg.pinv(phi_u)
    phi_v_pinv=np.linalg.pinv(phi_v)

    u_ref = u0[free_raveled_indicies]
    v_ref = v0[free_raveled_indicies]

    I_hat=np.eye(POD_basis)

    maxitr=10
    tol=1e-8
    
    Mphi_u=M.dot(phi_u)
    Nphi_u=N.dot(phi_u)
    Mphi_v=M.dot(phi_v)
    Nphi_v=N.dot(phi_v)

    Dxphi_u=Dx.dot(phi_u)
    Dyphi_u=Dy.dot(phi_u)
    Dxphi_v=Dx.dot(phi_v)
    Dyphi_v=Dy.dot(phi_v)

    au_cur=-1/dx*u_ref*(Mu_ref-Bdudx0_cur)-1/dy*v_ref*(Nu_ref-Bdudy0_cur) \
    +1/(Re*dx**2)*(Dxu_ref+bxu_cur)+1/(Re*dy**2)*(Dyu_ref+byu_cur)
    Bu_cur=-1/dx*(np.diag(u_ref).dot(Mphi_u)+np.diag(Mu_ref-Bdudx0_cur).dot(phi_u)) \
    -1/dy*np.diag(v_ref).dot(Nphi_u) \
    +1/(Re*dx**2)*Dxphi_u+1/(Re*dy**2)*Dyphi_u
    Cu_cur=-1/dy*np.diag(Nu_ref-Bdudy0_cur).dot(phi_v) 

    av_cur=-1/dx*u_ref*(Mv_ref-Bdvdx0_cur)-1/dy*v_ref*(Nv_ref-Bdvdy0_cur) \
    +1/(Re*dx**2)*(Dxv_ref+bxv_cur)+1/(Re*dy**2)*(Dyv_ref+byv_cur)
    Bv_cur=-1/dx*np.diag(Mv_ref-Bdvdx0_cur).dot(phi_u)
    Cv_cur=-1/dx*np.diag(u_ref).dot(Mphi_v) \
    -1/dy*(np.diag(v_ref).dot(Nphi_v)+np.diag(Nv_ref-Bdvdy0_cur).dot(phi_v)) \
    +1/(Re*dx**2)*Dxphi_v+1/(Re*dy**2)*Dyphi_v
    
    a1_cur=phi_u_pinv.dot(au_cur)
    A2_cur=phi_u_pinv.dot(Bu_cur)
    A3_cur=phi_u_pinv.dot(Cu_cur)

    b1_cur=phi_v_pinv.dot(av_cur)
    B2_cur=phi_v_pinv.dot(Bv_cur)
    B3_cur=phi_v_pinv.dot(Cv_cur)
    
    def theta_u(u_hat,v_hat):
        return -1/dx*phi_u.dot(u_hat)*Mphi_u.dot(u_hat)-1/dy*phi_v.dot(v_hat)*Nphi_u.dot(u_hat)

    def theta_v(u_hat,v_hat):
        return -1/dx*phi_u.dot(u_hat)*Mphi_v.dot(v_hat)-1/dy*phi_v.dot(v_hat)*Nphi_v.dot(v_hat)

    def dtheta_udu_hat(u_hat,v_hat):
        return -1/dx*(np.diag(Mphi_u.dot(u_hat)).dot(phi_u)+np.diag(phi_u.dot(u_hat)).dot(Mphi_u)) \
    -1/dy*(np.diag(phi_v.dot(v_hat)).dot(Nphi_u))

    def dtheta_udv_hat(u_hat,v_hat):
        return -1/dy*(np.diag(Nphi_u.dot(u_hat)).dot(phi_v))

    def dtheta_vdu_hat(u_hat,v_hat):
        return -1/dx*(np.diag(Mphi_v.dot(v_hat)).dot(phi_u))

    def dtheta_vdv_hat(u_hat,v_hat):
        return -1/dx*(np.diag(phi_u.dot(u_hat)).dot(Mphi_v)) \
    -1/dy*(np.diag(Nphi_v.dot(v_hat)).dot(phi_v)+np.diag(phi_v.dot(v_hat)).dot(Nphi_v))

    def f_u_hat(u_hat,v_hat,a1_cur,A2_cur,A3_cur):
        return a1_cur+A2_cur.dot(u_hat)+A3_cur.dot(v_hat)+phi_u_pinv.dot(theta_u(u_hat,v_hat))

    def f_v_hat(u_hat,v_hat,b1_cur,B2_cur,B3_cur):
        return b1_cur+B2_cur.dot(u_hat)+B3_cur.dot(v_hat)+phi_v_pinv.dot(theta_v(u_hat,v_hat))

    def df_u_hatdu_hat(u_hat,v_hat,A2_cur):
        return A2_cur+phi_u_pinv.dot(dtheta_udu_hat(u_hat,v_hat))

    def df_u_hatdv_hat(u_hat,v_hat,A3_cur):
        return A3_cur+phi_u_pinv.dot(dtheta_udv_hat(u_hat,v_hat))

    def df_v_hatdu_hat(u_hat,v_hat,B2_cur):
        return B2_cur+phi_v_pinv.dot(dtheta_vdu_hat(u_hat,v_hat))

    def df_v_hatdv_hat(u_hat,v_hat,B3_cur):
        return B3_cur+phi_v_pinv.dot(dtheta_vdv_hat(u_hat,v_hat))

    def r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,b1_cur,B2_cur,B3_cur):
        r_u_hat=u_hat-u_hat_prev-dt*f_u_hat(u_hat,v_hat,a1_cur,A2_cur,A3_cur)
        r_v_hat=v_hat-v_hat_prev-dt*f_v_hat(u_hat,v_hat,b1_cur,B2_cur,B3_cur)
        return np.concatenate((r_u_hat,r_v_hat))

    def J_hat(u_hat,v_hat,A2_cur,A3_cur,B2_cur,B3_cur):
        J_hat11=I_hat-dt*df_u_hatdu_hat(u_hat,v_hat,A2_cur)
        J_hat12=-dt*df_u_hatdv_hat(u_hat,v_hat,A3_cur)
        J_hat21=-dt*df_v_hatdu_hat(u_hat,v_hat,B2_cur)
        J_hat22=I_hat-dt*df_v_hatdv_hat(u_hat,v_hat,B3_cur)
        return np.block([[J_hat11,J_hat12],[J_hat21,J_hat22]])
    
    # solve ROM
    t_start_g=time.time()

    # solution
    u_rom_g=np.zeros((nt+1,ny*nx))
    v_rom_g=np.zeros((nt+1,ny*nx))
    u_reduced=np.zeros((nt+1,POD_basis))
    v_reduced=np.zeros((nt+1,POD_basis))

    # IC
    u_hat0=np.zeros(POD_basis)
    v_hat0=np.zeros(POD_basis)
    u_rom_g[0]=np.copy(u0)
    v_rom_g[0]=np.copy(v0)

    for k in range(nt):
    #     print("")
    #     print(k,"th time step:")

        u_hat_prev=np.copy(u_reduced[k])
        v_hat_prev=np.copy(v_reduced[k])

        u_hat=np.copy(u_reduced[k])
        v_hat=np.copy(v_reduced[k])

        residual_hat=r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,b1_cur,B2_cur,B3_cur)

        for itr in range(maxitr):
            Jacobian_hat=J_hat(u_hat,v_hat,A2_cur,A3_cur,B2_cur,B3_cur)

            delta_reduced=np.linalg.solve(Jacobian_hat,-residual_hat)

            u_hat+=delta_reduced[:POD_basis]
            v_hat+=delta_reduced[POD_basis:]

            residual_hat=r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,b1_cur,B2_cur,B3_cur)

            R=np.linalg.norm(residual_hat)
    #         print(itr,"th Newton iteration", "R:", R)

            if R<tol:
                u_reduced[k+1]=u_hat.copy()
                v_reduced[k+1]=v_hat.copy()
                u_rom_g[k+1,free_raveled_indicies]=u_ref+phi_u.dot(u_hat)
                v_rom_g[k+1,free_raveled_indicies]=v_ref+phi_v.dot(v_hat)
                # BC from exact solution
                u_rom_g[k+1,fixed_raveled_indicies]=u0[fixed_raveled_indicies].copy()
                v_rom_g[k+1,fixed_raveled_indicies]=v0[fixed_raveled_indicies].copy()
                break

        if R>=tol:
            print("\n non converged after {}th iteration".format(maxitr))
            break  

    #     u_reduced[k+1]=u_hat.copy()
    #     v_reduced[k+1]=v_hat.copy()
    #     u_rom_g[k+1,free_raveled_indicies]=u_ref+phi_u.dot(u_hat)
    #     v_rom_g[k+1,free_raveled_indicies]=v_ref+phi_v.dot(v_hat)
    #     # BC from exact solution
    #     u_rom_g[k+1,fixed_raveled_indicies]=u0[fixed_raveled_indicies].copy()
    #     v_rom_g[k+1,fixed_raveled_indicies]=v0[fixed_raveled_indicies].copy()

    t_elapsed_g=time.time()-t_start_g
    print()
    print('Time elapsed: {} sec'.format(t_elapsed_g))
    
    u_rel_err=np.linalg.norm(u_full-u_rom_g,ord=2,axis=1)/np.linalg.norm(u_full,ord=2,axis=1)*100
    v_rel_err=np.linalg.norm(v_full-v_rom_g,ord=2,axis=1)/np.linalg.norm(v_full,ord=2,axis=1)*100

    u_avg_rel_err=np.sqrt(np.sum(np.linalg.norm(u_full-u_rom_g,ord=2,axis=1)**2))/np.sqrt(np.sum(np.linalg.norm(u_full,ord=2,axis=1)**2))*100
    v_avg_rel_err=np.sqrt(np.sum(np.linalg.norm(v_full-v_rom_g,ord=2,axis=1)**2))/np.sqrt(np.sum(np.linalg.norm(v_full,ord=2,axis=1)**2))*100

    u_max_rel_err = np.max(u_rel_err)
    v_max_rel_err = np.max(v_rel_err)

    print("Average relative error of u: {} %".format(u_avg_rel_err))
    print("Average relative error of v: {} %".format(v_avg_rel_err))
    print("Maximum relative error of u: {} %".format(u_max_rel_err))
    print("Maximum relative error of v: {} %".format(v_max_rel_err))
    
    relErr_LSROM_g[:,ii]=[u_avg_rel_err,v_avg_rel_err]
    maxErr_LSROM_g[:,ii]=[u_max_rel_err,v_max_rel_err]

# LSPG

In [ ]:
for ii in range(len(nparamArr_LSROM)):
    nparam=nparamArr_LSROM[ii]
    
    # load POD basis
    if Re==10000:
        POD_basis=5
        file_name_POD="./model/POD_high_Re_nparam_{}.p".format(nparam)
    elif Re==100:
        POD_basis=5
        file_name_POD="./model/POD_low_Re_nparam_{}.p".format(nparam)
    else:
        raise NameError('{} is given for Re, but it must be either 100 or 10000'.format(Re))

    POD = pickle.load(open(file_name_POD,'rb'))

    phi_u = POD['u']
    phi_v = POD['v']
    
    phi_uT=phi_u.T
    phi_vT=phi_v.T

    u_ref = u0[free_raveled_indicies]
    v_ref = v0[free_raveled_indicies]

    I_hat=np.eye(POD_basis)

    maxitr=10
    tol=1e-8
    
    Mphi_u=M.dot(phi_u)
    Nphi_u=N.dot(phi_u)
    Mphi_v=M.dot(phi_v)
    Nphi_v=N.dot(phi_v)

    Mphi_uT=Mphi_u.T
    Nphi_uT=Nphi_u.T
    Mphi_vT=Mphi_v.T
    Nphi_vT=Nphi_v.T

    Dxphi_u=Dx.dot(phi_u)
    Dyphi_u=Dy.dot(phi_u)
    Dxphi_v=Dx.dot(phi_v)
    Dyphi_v=Dy.dot(phi_v)

    au_cur=-1/dx*u_ref*(Mu_ref-Bdudx0_cur)-1/dy*v_ref*(Nu_ref-Bdudy0_cur) \
    +1/(Re*dx**2)*(Dxu_ref+bxu_cur)+1/(Re*dy**2)*(Dyu_ref+byu_cur)
    Bu_cur=-1/dx*(np.diag(u_ref).dot(Mphi_u)+np.diag(Mu_ref-Bdudx0_cur).dot(phi_u)) \
    -1/dy*np.diag(v_ref).dot(Nphi_u) \
    +1/(Re*dx**2)*Dxphi_u+1/(Re*dy**2)*Dyphi_u
    Cu_cur=-1/dy*np.diag(Nu_ref-Bdudy0_cur).dot(phi_v)

    av_cur=-1/dx*u_ref*(Mv_ref-Bdvdx0_cur)-1/dy*v_ref*(Nv_ref-Bdvdy0_cur) \
    +1/(Re*dx**2)*(Dxv_ref+bxv_cur)+1/(Re*dy**2)*(Dyv_ref+byv_cur)
    Bv_cur=-1/dx*np.diag(Mv_ref-Bdvdx0_cur).dot(phi_u)
    Cv_cur=-1/dx*np.diag(u_ref).dot(Mphi_v) \
    -1/dy*(np.diag(v_ref).dot(Nphi_v)+np.diag(Nv_ref-Bdvdy0_cur).dot(phi_v)) \
    +1/(Re*dx**2)*Dxphi_v+1/(Re*dy**2)*Dyphi_v

    a1_cur=-dt*au_cur
    A2_cur=phi_u-dt*Bu_cur
    A3_cur=-dt*Cu_cur
    A2T_cur=A2_cur.T
    A3T_cur=A3_cur.T

    b1_cur=-dt*av_cur
    B2_cur=-dt*Bv_cur
    B3_cur=phi_v-dt*Cv_cur
    B2T_cur=B2_cur.T
    B3T_cur=B3_cur.T

    A4_cur=-A2_cur.T.dot(phi_u)
    A5_cur=-B2_cur.T.dot(phi_v)
    a6_cur=A2_cur.T.dot(a1_cur)+B2_cur.T.dot(b1_cur)
    A7_cur=A2_cur.T.dot(A2_cur)+B2_cur.T.dot(B2_cur)
    A8_cur=A2_cur.T.dot(A3_cur)+B2_cur.T.dot(B3_cur)

    B4_cur=-A3_cur.T.dot(phi_u)
    B5_cur=-B3_cur.T.dot(phi_v)
    b6_cur=A3_cur.T.dot(a1_cur)+B3_cur.T.dot(b1_cur)
    B7_cur=A3_cur.T.dot(A2_cur)+B3_cur.T.dot(B2_cur)
    B8_cur=A3_cur.T.dot(A3_cur)+B3_cur.T.dot(B3_cur)

    def theta_u(u_hat,v_hat):
        return -1/dx*phi_u.dot(u_hat)*Mphi_u.dot(u_hat)-1/dy*phi_v.dot(v_hat)*Nphi_u.dot(u_hat)

    def theta_v(u_hat,v_hat):
        return -1/dx*phi_u.dot(u_hat)*Mphi_v.dot(v_hat)-1/dy*phi_v.dot(v_hat)*Nphi_v.dot(v_hat)

    def dtheta_udu_hat(u_hat,v_hat):
        return -1/dx*(np.diag(Mphi_u.dot(u_hat)).dot(phi_u)+np.diag(phi_u.dot(u_hat)).dot(Mphi_u)) \
    -1/dy*(np.diag(phi_v.dot(v_hat)).dot(Nphi_u))

    def dtheta_udv_hat(u_hat,v_hat):
        return -1/dy*(np.diag(Nphi_u.dot(u_hat)).dot(phi_v))

    def dtheta_vdu_hat(u_hat,v_hat):
        return -1/dx*(np.diag(Mphi_v.dot(v_hat)).dot(phi_u))

    def dtheta_vdv_hat(u_hat,v_hat):
        return -1/dx*(np.diag(phi_u.dot(u_hat)).dot(Mphi_v)) \
    -1/dy*(np.diag(Nphi_v.dot(v_hat)).dot(phi_v)+np.diag(phi_v.dot(v_hat)).dot(Nphi_v))

    def dtheta_udu_hatT(u_hat,v_hat):
        return -1/dx*(Mphi_u.dot(u_hat)*phi_uT+phi_u.dot(u_hat)*Mphi_uT) \
    -1/dy*(phi_v.dot(v_hat)*Nphi_uT)

    def dtheta_udv_hatT(u_hat,v_hat):
        return -1/dy*(Nphi_u.dot(u_hat)*phi_vT)

    def dtheta_vdu_hatT(u_hat,v_hat):
        return -1/dx*(Mphi_v.dot(v_hat)*phi_uT)

    def dtheta_vdv_hatT(u_hat,v_hat):
        return -1/dx*(phi_u.dot(u_hat)*Mphi_vT) \
    -1/dy*(Nphi_v.dot(v_hat)*phi_v.T+phi_v.dot(v_hat)*Nphi_vT)

    def r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,A2T_cur,A3T_cur,A4_cur,A5_cur,a6_cur,A7_cur,A8_cur,\
              b1_cur,B2_cur,B3_cur,B2T_cur,B3T_cur,B4_cur,B5_cur,b6_cur,B7_cur,B8_cur):

        r_u_hat=A4_cur.dot(u_hat_prev)+A5_cur.dot(v_hat_prev)+a6_cur+A7_cur.dot(u_hat)+A8_cur.dot(v_hat) \
        -dt*A2T_cur.dot(theta_u(u_hat,v_hat))-dt*B2T_cur.dot(theta_v(u_hat,v_hat)) \
        -dt*dtheta_udu_hatT(u_hat,v_hat).dot(-phi_u.dot(u_hat_prev)+a1_cur+A2_cur.dot(u_hat)+A3_cur.dot(v_hat)-dt*theta_u(u_hat,v_hat)) \
        -dt*dtheta_vdu_hatT(u_hat,v_hat).dot(-phi_v.dot(v_hat_prev)+b1_cur+B2_cur.dot(u_hat)+B3_cur.dot(v_hat)-dt*theta_v(u_hat,v_hat))

        r_v_hat=B4_cur.dot(u_hat_prev)+B5_cur.dot(v_hat_prev)+b6_cur+B7_cur.dot(u_hat)+B8_cur.dot(v_hat) \
        -dt*A3T_cur.dot(theta_u(u_hat,v_hat))-dt*B3T_cur.dot(theta_v(u_hat,v_hat)) \
        -dt*dtheta_udv_hatT(u_hat,v_hat).dot(-phi_u.dot(u_hat_prev)+a1_cur+A2_cur.dot(u_hat)+A3_cur.dot(v_hat)-dt*theta_u(u_hat,v_hat)) \
        -dt*dtheta_vdv_hatT(u_hat,v_hat).dot(-phi_v.dot(v_hat_prev)+b1_cur+B2_cur.dot(u_hat)+B3_cur.dot(v_hat)-dt*theta_v(u_hat,v_hat))

        return np.concatenate((r_u_hat,r_v_hat))

    def J_hat(u_hat,v_hat,A2_cur,A2T_cur,A3_cur,A3T_cur,A7_cur,A8_cur,B2_cur,B2T_cur,B3_cur,B3T_cur,B7_cur,B8_cur):

        J_hat11=A7_cur-dt*A2T_cur.dot(dtheta_udu_hat(u_hat,v_hat))-dt*B2T_cur.dot(dtheta_vdu_hat(u_hat,v_hat)) \
        -dt*dtheta_udu_hatT(u_hat,v_hat).dot(A2_cur-dt*dtheta_udu_hat(u_hat,v_hat)) \
        -dt*dtheta_vdu_hatT(u_hat,v_hat).dot(B2_cur-dt*dtheta_vdu_hat(u_hat,v_hat))

        J_hat12=A8_cur-dt*A2T_cur.dot(dtheta_udv_hat(u_hat,v_hat))-dt*B2T_cur.dot(dtheta_vdv_hat(u_hat,v_hat)) \
        -dt*dtheta_udu_hatT(u_hat,v_hat).dot(A3_cur-dt*dtheta_udv_hat(u_hat,v_hat)) \
        -dt*dtheta_vdu_hatT(u_hat,v_hat).dot(B3_cur-dt*dtheta_vdv_hat(u_hat,v_hat))

        J_hat21=B7_cur-dt*A3T_cur.dot(dtheta_udu_hat(u_hat,v_hat))-dt*B3T_cur.dot(dtheta_vdu_hat(u_hat,v_hat)) \
        -dt*dtheta_udv_hatT(u_hat,v_hat).dot(A2_cur-dt*dtheta_udu_hat(u_hat,v_hat)) \
        -dt*dtheta_vdv_hatT(u_hat,v_hat).dot(B2_cur-dt*dtheta_vdu_hat(u_hat,v_hat))

        J_hat22=B8_cur-dt*A3T_cur.dot(dtheta_udv_hat(u_hat,v_hat))-dt*B3T_cur.dot(dtheta_vdv_hat(u_hat,v_hat)) \
        -dt*dtheta_udv_hatT(u_hat,v_hat).dot(A3_cur-dt*dtheta_udv_hat(u_hat,v_hat)) \
        -dt*dtheta_vdv_hatT(u_hat,v_hat).dot(B3_cur-dt*dtheta_vdv_hat(u_hat,v_hat))

        return np.block([[J_hat11,J_hat12],[J_hat21,J_hat22]])
    
    # run ROM
    t_start_lspg=time.time()

    # solution
    u_rom_lspg=np.zeros((nt+1,ny*nx))
    v_rom_lspg=np.zeros((nt+1,ny*nx))
    u_reduced=np.zeros((nt+1,POD_basis))
    v_reduced=np.zeros((nt+1,POD_basis))

    # IC
    u_hat0=np.zeros(POD_basis)
    v_hat0=np.zeros(POD_basis)
    u_rom_lspg[0]=np.copy(u0)
    v_rom_lspg[0]=np.copy(v0)

    for k in range(nt):
    #     print("")
    #     print(k+1,"th time step:")

        u_hat_prev=np.copy(u_reduced[k])
        v_hat_prev=np.copy(v_reduced[k])

        u_hat=np.copy(u_reduced[k])
        v_hat=np.copy(v_reduced[k])

        residual_hat=r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,A2T_cur,A3T_cur,A4_cur,A5_cur,a6_cur,A7_cur,A8_cur,\
              b1_cur,B2_cur,B3_cur,B2T_cur,B3T_cur,B4_cur,B5_cur,b6_cur,B7_cur,B8_cur)

        for itr in range(maxitr):
            Jacobian_hat=J_hat(u_hat,v_hat,A2_cur,A2T_cur,A3_cur,A3T_cur,A7_cur,A8_cur,B2_cur,B2T_cur,B3_cur,B3T_cur,B7_cur,B8_cur)

            delta_reduced=np.linalg.solve(Jacobian_hat,-residual_hat)

            u_hat+=delta_reduced[:POD_basis]
            v_hat+=delta_reduced[POD_basis:]

            residual_hat=r_hat(u_hat,v_hat,u_hat_prev,v_hat_prev,a1_cur,A2_cur,A3_cur,A2T_cur,A3T_cur,A4_cur,A5_cur,a6_cur,A7_cur,A8_cur,\
              b1_cur,B2_cur,B3_cur,B2T_cur,B3T_cur,B4_cur,B5_cur,b6_cur,B7_cur,B8_cur)

            R=np.linalg.norm(residual_hat)
    #         print(itr,"th Newton iteration", "R:", R)

            if R<tol:
                u_reduced[k+1]=u_hat.copy()
                v_reduced[k+1]=v_hat.copy()
                u_rom_lspg[k+1,free_raveled_indicies]=u_ref+phi_u.dot(u_hat)
                v_rom_lspg[k+1,free_raveled_indicies]=v_ref+phi_v.dot(v_hat)
                # BC from exact solution
                u_rom_lspg[k+1,fixed_raveled_indicies]=u0[fixed_raveled_indicies].copy()
                v_rom_lspg[k+1,fixed_raveled_indicies]=v0[fixed_raveled_indicies].copy()
                break

        if R>=tol:
            print("\n non converged after {}th iteration".format(maxitr))
            break  

    #     u_reduced[k+1]=u_hat.copy()
    #     v_reduced[k+1]=v_hat.copy()
    #     u_rom_lspg[k+1,free_raveled_indicies]=u_ref+phi_u.dot(u_hat)
    #     v_rom_lspg[k+1,free_raveled_indicies]=v_ref+phi_v.dot(v_hat)
    #     # BC from exact solution
    #     u_rom_lspg[k+1,fixed_raveled_indicies]=u[k+1,fixed_raveled_indicies].copy()
    #     v_rom_lspg[k+1,fixed_raveled_indicies]=v[k+1,fixed_raveled_indicies].copy()

    t_elapsed_lspg=time.time()-t_start_lspg
    print()
    print('Time elapsed: {} sec'.format(t_elapsed_lspg))
    
    u_rel_err=np.linalg.norm(u_full-u_rom_lspg,ord=2,axis=1)/np.linalg.norm(u_full,ord=2,axis=1)*100
    v_rel_err=np.linalg.norm(v_full-v_rom_lspg,ord=2,axis=1)/np.linalg.norm(v_full,ord=2,axis=1)*100

    u_avg_rel_err=np.sqrt(np.sum(np.linalg.norm(u_full-u_rom_lspg,ord=2,axis=1)**2))/np.sqrt(np.sum(np.linalg.norm(u_full,ord=2,axis=1)**2))*100
    v_avg_rel_err=np.sqrt(np.sum(np.linalg.norm(v_full-v_rom_lspg,ord=2,axis=1)**2))/np.sqrt(np.sum(np.linalg.norm(v_full,ord=2,axis=1)**2))*100

    u_max_rel_err = np.max(u_rel_err)
    v_max_rel_err = np.max(v_rel_err)

    print("Average relative error of u: {} %".format(u_avg_rel_err))
    print("Average relative error of v: {} %".format(v_avg_rel_err))
    print("Maximum relative error of u: {} %".format(u_max_rel_err))
    print("Maximum relative error of v: {} %".format(v_max_rel_err))
    
    relErr_LSROM_lspg[:,ii]=[u_avg_rel_err,v_avg_rel_err]
    maxErr_LSROM_lspg[:,ii]=[u_max_rel_err,v_max_rel_err]   

# Plot errors

In [ ]:
# plt.figure()
# plt.plot(nparamArr_LSROM,relErr_LSROM_g[0],nparamArr_LSROM,relErr_LSROM_lspg[0],nparamArr_LSROM,proErr_LSROM[0])
# plt.legend(['LS-ROM-g','LS-ROM-lspg','LS proj. error'])
# plt.xlabel('Number of parameter instances')
# plt.ylabel('Avg. rel. error of u (%)')
# plt.show()

# plt.figure()
# plt.plot(nparamArr_LSROM,relErr_LSROM_g[1],nparamArr_LSROM,relErr_LSROM_lspg[1],nparamArr_LSROM,proErr_LSROM[1])
# plt.legend(['LS-ROM-g','LS-ROM-lspg','LS proj. error'])
# plt.xlabel('Number of parameter instances')
# plt.ylabel('Avg. rel. error of v (%)')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(nparamArr_LSROM,maxErr_LSROM_g[0],nparamArr_LSROM,maxErr_LSROM_lspg[0],nparamArr_LSROM,proErr_LSROM[0])
# plt.legend(['LS-ROM-g','LS-ROM-lspg','LS proj. error'])
# plt.xlabel('Number of parameter instances')
# plt.ylabel('Max. rel. error of u (%)')
# plt.show()

# plt.figure()
# plt.plot(nparamArr_LSROM,maxErr_LSROM_g[1],nparamArr_LSROM,maxErr_LSROM_lspg[1],nparamArr_LSROM,proErr_LSROM[1])
# plt.legend(['LS-ROM-g','LS-ROM-lspg','LS proj. error'])
# plt.xlabel('Number of parameter instances')
# plt.ylabel('Max. rel. error of v (%)')
# plt.show()

# Save data to mat file

In [ ]:
dict_err={"relErr_LSROM_g":relErr_LSROM_g,"relErr_LSROM_lspg":relErr_LSROM_lspg,\
          "maxErr_LSROM_g":maxErr_LSROM_g,"maxErr_LSROM_lspg":maxErr_LSROM_lspg,\
          'proErr_LSROM':proErr_LSROM,'nparamArr_LSROM':nparamArr_LSROM}
savemat("./data/LSROM_Err.mat",dict_err)

pickle.dump(dict_err,open('./data/LSROM_Err.p','wb'))